In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re, pickle
le = LabelEncoder()
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to C:\Users\SRAVANTHI
[nltk_data]     SHOROFF\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SRAVANTHI
[nltk_data]     SHOROFF\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\SRAVANTHI
[nltk_data]     SHOROFF\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_excel(r"SCOM.xlsx")
df.head()

,Alert,Severity,Server,Action,Alert Type(ignorable)
0,CL - Service Account Lockout,Very Low,any server,1. Look for the account locked out name from A...,Yes
1,KCC cannot compute a replication path,Very Low,any server,"Ignore the alert, if it is excessively repeati...",yes
2,"AD,Mail,Lync alerts",Medium,any server,p2 case to WP server application team and drop...,doubt(needs discussion)
3,Backup device failed - Operating system error,Low,NaN,Drop a mail to Wintel team and backup team,No
4,IIS 7 Web site is unavailable,Critical,any server,Drop a mail to Wintel team,NO


In [4]:
df.keys()

Index(['Alert', 'Severity', 'Server', 'Action', 'Alert Type(ignorable)'], dtype='object')

In [5]:
df['Alert'].dropna(how='all')

0                           CL - Service Account Lockout
1                  KCC cannot compute a replication path
2                                    AD,Mail,Lync alerts
3          Backup device failed - Operating system error
4                          IIS 7 Web site is unavailable
                             ...                        
327                         \nCertificate lifespan alert
328    Failed to ping or bind to the Domain Naming Ma...
329         Folder Redirection CSE Processed With Errors
330      Application pool worker process is unresponsive
331    Exchange Control Panel connectivity (External)...
Name: Alert, Length: 332, dtype: object

In [6]:
df['Alert Type(ignorable)'].dropna(how='all')

0                          Yes
1                          yes
2      doubt(needs discussion)
3                           No
4                           NO
                ...           
327                         no
328                        yes
329                        yes
330                         no
331                         no
Name: Alert Type(ignorable), Length: 330, dtype: object

In [7]:
# preprocessing
df['Alert Type(ignorable)'] = df['Alert Type(ignorable)'].replace(to_replace=['NO','no','No(needs discussion)'],value='No')
df['Alert Type(ignorable)'] = df['Alert Type(ignorable)'].replace(to_replace=['yes','yes(server decom)','yes( server decom)'],value='Yes')

In [10]:
df['Alert Type(ignorable)'].value_counts()

Yes    173
No     151
Name: Alert Type(ignorable), dtype: int64

In [9]:
df.drop(df.loc[df['Alert Type(ignorable)'] == 'doubt(needs discussion)'].index,inplace=True)
df.dropna(subset=['Alert Type(ignorable)'],inplace = True)

In [11]:
df['Alert'] = df['Alert'].apply(lambda x: x.lower())
df['Alert'] = df['Alert'].apply(lambda s: re.sub(r"[^a-zA-Z0-9]"," ",s))

In [12]:
# lemmetization
lemmatizer=WordNetLemmatizer()

In [14]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens =[w for w in tokens if not w in stop] # [w for w in
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
    return lemmatized_output



In [15]:
df['Alert'].apply(tokenize)

0                             cl service account lockout
1                           kcc compute replication path
3            backup device failed operating system error
4                              ii 7 web site unavailable
5      sql server database configuration monitoring d...
                             ...                        
327                           certificate lifespan alert
328    failed ping bind domain naming master fsmo rol...
329               folder redirection cse processed error
330         application pool worker process unresponsive
331    exchange control panel connectivity external t...
Name: Alert, Length: 324, dtype: object

In [17]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [24]:
le = preprocessing.LabelEncoder()
alerts = le.fit_transform(df['Alert'])

In [56]:
filename = 'labelencoder.pkl'
pickle.dump(alerts, open('labelencoder.pkl', 'wb'))

In [57]:
tf = pickle.load(open("labelencoder.pkl", "rb"))

In [58]:
tf

array([ 51,   0,  39, 133, 244,  74,   9,  93, 135, 158,  63, 113,  80,
        87, 154, 180, 178, 121, 303, 252, 223,  83, 125, 231, 211, 200,
        30, 261,   6,   1, 169, 182, 167, 149, 154, 155, 127, 260, 313,
         8,  70,  32, 151,  64,  62, 229, 302, 143,  53,  96, 122, 123,
       124, 126, 129, 130, 131,  52, 120,  98, 118, 119,  61, 128, 117,
        75, 176,  84, 278, 264,   8, 181,  17,  19,  23,  24,  36,  38,
        20,  21,  50,  68, 103, 106,  95, 116, 165, 161, 217, 222, 240,
       272, 286, 311, 316, 315, 187, 188, 189, 215, 258, 273, 281, 290,
       291, 297, 314, 230,   7,  31, 268,  26, 267, 153, 289, 292, 288,
       285, 280, 274, 262, 248, 233, 226, 221, 220, 202, 185, 166, 162,
       157, 156, 145, 142, 140, 132,  91,  55,  46,  18,  29,  28,  14,
        11, 296, 206, 295, 173, 201, 254, 172, 171, 100,  99,  97,  79,
       186,  58, 190, 191, 192, 193, 194, 195, 196, 197, 232, 107, 177,
        81, 298, 246, 178, 137,  12,  13,  16,  22,  34,  47,  5

In [68]:
X = tf.reshape(-1,1)
y = df['Alert Type(ignorable)']
print(X)

[[ 51]
 [  0]
 [ 39]
 [133]
 [244]
 [ 74]
 [  9]
 [ 93]
 [135]
 [158]
 [ 63]
 [113]
 [ 80]
 [ 87]
 [154]
 [180]
 [178]
 [121]
 [303]
 [252]
 [223]
 [ 83]
 [125]
 [231]
 [211]
 [200]
 [ 30]
 [261]
 [  6]
 [  1]
 [169]
 [182]
 [167]
 [149]
 [154]
 [155]
 [127]
 [260]
 [313]
 [  8]
 [ 70]
 [ 32]
 [151]
 [ 64]
 [ 62]
 [229]
 [302]
 [143]
 [ 53]
 [ 96]
 [122]
 [123]
 [124]
 [126]
 [129]
 [130]
 [131]
 [ 52]
 [120]
 [ 98]
 [118]
 [119]
 [ 61]
 [128]
 [117]
 [ 75]
 [176]
 [ 84]
 [278]
 [264]
 [  8]
 [181]
 [ 17]
 [ 19]
 [ 23]
 [ 24]
 [ 36]
 [ 38]
 [ 20]
 [ 21]
 [ 50]
 [ 68]
 [103]
 [106]
 [ 95]
 [116]
 [165]
 [161]
 [217]
 [222]
 [240]
 [272]
 [286]
 [311]
 [316]
 [315]
 [187]
 [188]
 [189]
 [215]
 [258]
 [273]
 [281]
 [290]
 [291]
 [297]
 [314]
 [230]
 [  7]
 [ 31]
 [268]
 [ 26]
 [267]
 [153]
 [289]
 [292]
 [288]
 [285]
 [280]
 [274]
 [262]
 [248]
 [233]
 [226]
 [221]
 [220]
 [202]
 [185]
 [166]
 [162]
 [157]
 [156]
 [145]
 [142]
 [140]
 [132]
 [ 91]
 [ 55]
 [ 46]
 [ 18]
 [ 29]
 [ 28]
 [ 14]

In [69]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [70]:
glmMod = LogisticRegression(penalty='l1', dual=False, tol=0.0001, C=1.0, fit_intercept=True,
                            intercept_scaling=1, class_weight=None, 
                            random_state=None, solver='liblinear', max_iter=100,
                            multi_class='ovr', verbose=2)
glmMod.fit(X_train, y_train)
y_pred=glmMod.predict(X_test)
print("\n\n***** Logistic Regression classification report:*****\n")
print(classification_report(y_test, y_pred))

[LibLinear]

***** Logistic Regression classification report:*****

              precision    recall  f1-score   support

          No       0.43      0.35      0.39        34
         Yes       0.58      0.66      0.62        47

    accuracy                           0.53        81
   macro avg       0.51      0.51      0.50        81
weighted avg       0.52      0.53      0.52        81



In [32]:
filename = 'logisticMod.pkl'
pickle.dump(glmMod, open('logisticMod.pkl', 'wb'))

In [33]:
rfMod = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2,
                               min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',
                               max_leaf_nodes=None, bootstrap=True, oob_score=False, n_jobs=1, 
                               random_state=None, verbose=0)
rfMod.fit(X_train, y_train)
y_pred=rfMod.predict(X_test)
print("\n\n***** RandomForestClassifier report:*****\n")
print(classification_report(y_test, y_pred))



***** RandomForestClassifier report:*****

              precision    recall  f1-score   support

          No       0.56      0.69      0.62        32
         Yes       0.76      0.65      0.70        49

    accuracy                           0.67        81
   macro avg       0.66      0.67      0.66        81
weighted avg       0.68      0.67      0.67        81



In [35]:
filename = 'randomforestMod.pkl'
pickle.dump(rfMod, open('randomforestMod.pkl', 'wb'))

In [36]:
gbMod = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200, subsample=1.0,
                                   min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                   max_depth=3,
                                   init=None, random_state=None, max_features=None, verbose=0)
gbMod.fit(X_train, y_train)
y_pred=gbMod.predict(X_test)
print("\n\n***** GradientBoosting report:*****\n")
print(classification_report(y_test, y_pred))



***** GradientBoosting report:*****

              precision    recall  f1-score   support

          No       0.59      0.59      0.59        32
         Yes       0.73      0.73      0.73        49

    accuracy                           0.68        81
   macro avg       0.66      0.66      0.66        81
weighted avg       0.68      0.68      0.68        81



In [49]:
filename = 'gradientboostMod1.pkl'
pickle.dump(gbMod, open('gradientboostMod.pkl', 'wb'))

In [50]:
adaMod = AdaBoostClassifier(base_estimator=None, n_estimators=200, learning_rate=1.0)
adaMod.fit(X_train, y_train)
y_pred = adaMod.predict(X_test)
print("\n\n***** Adaboost report:*****\n")
print(classification_report(y_test, y_pred))



***** Adaboost report:*****

              precision    recall  f1-score   support

          No       0.58      0.47      0.52        32
         Yes       0.69      0.78      0.73        49

    accuracy                           0.65        81
   macro avg       0.63      0.62      0.62        81
weighted avg       0.65      0.65      0.65        81



In [51]:
filename = 'adaboostMod1.pkl'
pickle.dump(adaMod, open('adaboostMod1.pkl', 'wb'))

In [52]:
from sklearn.ensemble import VotingClassifier
votingMod = VotingClassifier(estimators=[('RandomForrest', rfMod),('LogisticRegression', glmMod),("GradientBoosting",gbMod),("AdaBoost",adaMod)], voting='soft')
votingMod = votingMod.fit(X_train, y_train)
test_labels=votingMod.predict((X_test))
#votingMod.score(X_test_transform, y_test)

print("\n\n***** Model Ensemble score report:*****\n")
print(classification_report(y_test, test_labels))


[LibLinear]

***** Model Ensemble score report:*****

              precision    recall  f1-score   support

          No       0.59      0.62      0.61        32
         Yes       0.74      0.71      0.73        49

    accuracy                           0.68        81
   macro avg       0.67      0.67      0.67        81
weighted avg       0.68      0.68      0.68        81



In [53]:
filename = 'votingclassifierMod1.pkl'
pickle.dump(votingMod, open('votingclassifierMod1.pkl', 'wb'))

In [54]:
test_labels.size

81

In [55]:
pwd

'C:\\Users\\SRAVANTHI SHOROFF\\Desktop\\sravanthi\\SCOM_dump_ML'